### CNN network를 활용하여 이미지간 유사도
: pixel값을 column으로 하여 유사도를 구한다.
1. training을 시킨 model의 _dense를 추출하여 유사도를 구한다.
2. CNN을 통과시켜 feature의 차원이 축소된 이미지 특성값으로 유사도를 구한다.

In [4]:
import tensorflow as tf
from PIL import Image as pil
from scipy.misc import imread, imresize
import os
from scipy import ndimage
from six.moves import urllib

cwd = os.getcwd()
cwd

'C:\\Users\\Juno\\Desktop\\juno_project\\Webtoon-Recommender_System'

In [89]:
# 웹툰 labeling을 위한 dictionary를 만든다.

# Load info
info = pd.read_csv("data_Juno/webtoon_naver_info_by_genres.csv")

# make dict
dic = {}
for i, id in enumerate(info.unique_id):
    dic[id] = i

path = cwd + "./0_img/main_image_naver/color/"
print(path)
flist = os.listdir(path)
n_class = len(dic)
imgcnt = 0
for f in flist:
    fullpath = os.path.join(path,f)
    currimg = np.asarray(pil.open(fullpath).convert("RGB"))
    img = currimg / 255.
#     print(img.shape)
#     print(imresize(img, (64,64)).shape)
#     print(img)
    img_r = imresize(img, (32,32))
    imgvec = np.reshape(img_r, (1,-1))
    
    # make labels
    id_check = int(f.split(".")[0])
    label = dic[id_check]
    curr_label = np.eye(n_class, n_class)[label:label+1, :]
    if imgcnt == 0:
        totalimg = imgvec
        totallabel = curr_label
    else:
        totalimg = np.concatenate((totalimg, imgvec), axis=0)
        totallabel = np.concatenate((totallabel, curr_label), axis=0)
    print("now: %s finished!!:" % imgcnt, end='\r')
    imgcnt += 1
print ("Total %d images loaded." % (imgcnt))

C:\Users\Juno\Desktop\juno_project\Webtoon-Recommender_System./0_img/main_image_naver/color/


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Total 745 images loaded.


In [5]:
n_input = 3072
n_channel = 32 # convolution filter의 갯수
n_classes = 745
n_imgch = 3
x = tf.placeholder(dtype='float', shape=[None, n_input])
y = tf.placeholder(dtype='float', shape=[None, n_classes])

stddev = 0.1
# convolution filters
weights = {
    'c1' : tf.Variable(initial_value=tf.random_normal(shape=[3,3,n_imgch,n_channel], stddev=stddev)),
    'c2' : tf.Variable(initial_value=tf.random_normal(shape=[3,3,n_channel,32], stddev=stddev)),
    'c3' : tf.Variable(initial_value=tf.random_normal(shape=[3,3,32,32], stddev=stddev)),
    'd1' : tf.Variable(initial_value=tf.random_normal(shape=[4*4*32,n_classes], stddev=stddev)),
}
biases = {
    'c1' : tf.Variable(initial_value=tf.random_normal(shape=[n_channel], stddev=stddev)),
    'c2' : tf.Variable(initial_value=tf.random_normal(shape=[32], stddev=stddev)),
    'c3' : tf.Variable(initial_value=tf.random_normal(shape=[32], stddev=stddev)),
    'd1' : tf.Variable(initial_value=tf.random_normal(shape=[n_classes], stddev=stddev)),
}
print("Model is defined!!")

Model is defined!!


In [6]:
def CNN(_x, _w, _b):
    # Reshape
    _x_r = tf.reshape(_x, shape=[-1,32,32,3])
    # Convolution
    _conv1 = tf.nn.conv2d(_x_r, _w['c1'], strides=[1,1,1,1], padding='SAME') # zero-padding
    # add bias
    _conv2 = tf.nn.bias_add(_conv1, _b['c1'])
    # Passing the ReLU
    _conv3 = tf.nn.relu(_conv2)
    # max pooling
    # non-overlapping
    _pool = tf.nn.max_pool(_conv3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    # 2nd Convolutional Layer
    _second = tf.nn.conv2d(_pool, _w['c2'], strides=[1,1,1,1], padding='SAME')
    _second = tf.nn.bias_add(_second, _b['c2'])
    _second = tf.nn.relu(_second)
    _second = tf.nn.max_pool(_second, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    # 3rd Convolutional Layer
    _third = tf.nn.conv2d(_second, _w['c3'], strides=[1,1,1,1], padding='SAME')
    _third = tf.nn.bias_add(_third, _b['c3'])
    _third = tf.nn.relu(_third)
    _third = tf.nn.max_pool(_third, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    # fully connected layer, 한 줄로 쫙 편다.
    _dense = tf.reshape(_third, shape=[-1,  _w['d1'].get_shape().as_list()[0]])
    _logit = tf.add(tf.matmul(_dense, _w['d1']),_b['d1'])
    
    # Return to dict type
    _out = {
        'x_r':_x_r, 'conv1':_conv1, 'conv2' : _conv2, 'conv3': _conv3,
        'pool': _pool, 'dense': _dense, 'logit':_logit
    }
    return _out

In [13]:
cnnout = CNN(x, weights, biases)
features = cnnout['dense']

**1. training을 시킨 model의 _dense를 추출하여 유사도를 구한다.**

In [15]:
savedir = "./3_NeuralNet_image/main_image/"
saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
best_epoch = 9
restorename = savedir + "net-" + str(best_epoch) + ".ckpt"
print ("LOADING [%s]" % (restorename))
saver.restore(sess, restorename)
result = sess.run(features, feed_dict={x:totalimg})


LOADING [./3_NeuralNet_image/main_image/net-9.ckpt]
INFO:tensorflow:Restoring parameters from ./3_NeuralNet_image/main_image/net-9.ckpt


In [57]:
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity, euclidean_distances

In [39]:
result[0].reshape((1,-1)).shape, result.shape

((1, 512), (745, 512))

In [60]:
cosine_similar = linear_kernel(result[0].reshape((1,-1)), result).flatten()
# 위 코드는 아래와 같은 역할을 한다
# cosine_similar = (srch_vector*X.T).toarray().flatten()

# 유사한 rawdata index
sim_rank_idx = cosine_similar.argsort()[::-1]
sim_rank_idx[:5]

array([498,  74,  73, 226,  51], dtype=int64)

In [62]:
cosine_similar = euclidean_distances(result[1].reshape((1,-1)), result).flatten()
sim_rank_idx = cosine_similar.argsort()
sim_rank_idx[:5]

array([  1, 622, 552, 676, 593], dtype=int64)

---
**2. CNN을 통과시켜 feature의 차원이 축소된 이미지 특성값으로 유사도를 구한다.**

In [91]:
total_image = np.array(totalimg, dtype='float32') / 255.

In [92]:
total_image.shape

(745, 3072)

In [96]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [97]:
cnnout = CNN(x, weights, biases)
features = cnnout['dense']
result = sess.run(features, feed_dict={x:total_image})

In [99]:
result.shape

(745, 512)

In [106]:
euclidean_dist = euclidean_distances(result[1].reshape((1,-1)), result).flatten()
sim_rank_idx = euclidean_dist.argsort()
sim_rank_idx[:10]

array([  1,   0, 325, 552, 651, 250, 167, 516, 160, 432], dtype=int64)

In [101]:
dic2 = {}
for k, v in dic.items():
    dic2[v] = k

In [107]:
dic2[250]

670143

**둘 다 크게 좋은 결과를 도출해내지 못하였다.**